# I. Introduction

# II. Background

# III. How to Use

# IV. Software Organization

# V. Implementation

## Class 1: AD

Describes variables in 1-Dimension

#### Attributes
- val: float
    - A numeric value, indicating the value of the current AD instance.
    
- der: array of float
    - Suppose there are m elementary variables, 'der' would be an array of m elements, each representing the derivative value with respect to a certain elementary variable.
    - We store these values in an array so that when our later computation is easier.

#### Methods
0. \__init__:
    - arguments: 
        - val: float, representing the value
        - der: a list of floats, representing the derivatives with respect to elementary variables; optional, default=1
    - sets self.val=val, self.der=der


1. \__add__ & \__radd__: 
    - arguments:
        - self
        - other: a float, int, or AD
    - returns: 
        - if other is an AD -> a new AD instance with new.val = self.val + other.val, new.der = self.der + other.der
        - if other is a numeric value -> a new AD instance with new.val = self.val + other, new.der = self.der


2. \__sub__ & \__rsub__
    - arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val - other.val, new.der = self.der - other.der
		- if other is a numeric value -> a new AD instance with new.val = self.val - other, new.der = self.der


3. \__mul__ & \__rmul__
	- arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val \* other.val, new.der = self.val \* other.der + self.der \* other.val
		- if other is a numeric value -> a new AD instance with new.val = self.val \* other, new.der = self.der \* other


4. \__div__ & \__rdiv__
	- arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val / other.val, new.der = (self.der \* other.val + self.val \* other.der)/(other.val\*\*2)
		- if other is a numeric value -> a new AD instance with new.val = self.val / other, new.der = self.der / other
	- raises:
		- ZeroDivisionError when other.val = 0


5. \__pow__
	- arguments:
		- self
		- k: a float, int, or AD
	- returns:
		- if other is an AD -> a new AD instance with new.val = self.val \*\* other.val, new.der = other.val \* self.val \*\* (other.val - 1) \* self.der + log(self.val) \* self.val \*\* other.val \* other.der
		- if other is a numeric value -> a new AD instance with new.val = self.val \*\* k, new.der = k \* (self.val\*\*(k-1)) \* self.der


6. \__abs__
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = abs(self.val), new.der = sign(self.val) \* self.der


7. sin
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = sin(self.val), new.der = cos(self.val) \* self.der


8. cos
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = cos(self.val), new.der = -sin(self.val) \* self.der


9. tan
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = cos(self.val), new.der = 1/(cos(self.val) \*\* 2) \* self.der


10. exp
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = exp(self.val), new.der = exp(self.val) \* self.der


11. log
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = log(self.val), new.der = self.der/self.val
	- raises: (????)
		exception when self.val <= 0 

12. get_val()
    - arguments:
        - self
    - returns:
        - self.val

13. get_der(i=1)
    - arguments:
        - self
    - returns:
        - self.der

14. \__eq__
    - arguments:
        - self
        - other: an AD instance
    - returns:
        - 'True' if self.val==other.val and self.der==other.der, 'False' otherwise
        
15. \__str__
    - arguments:
        - self
    - returns:
        - a string describing the value and derivatives of the current instance

## Class 2: AD_System

Describes variables in n-Dimension

#### Attributes
- val: array of float
    - Numeric values, indicating the value of each entry in the current AD_System instance.
    
- der: 2D array of float
    - Suppose there are m elementary variables and n variables in the AD_System, 'der' would be a n\*m array of elements, with the (i,j) entry representing the derivative value of the i-th AD with respect to the j-th elementary variable.

#### Methods
0. \__init__:
    - arguments: 
        - a list/array of AD instances
    - sets self.val as a list of 'val' attributes of the input AD instances
    - combines the 'der' attributes of the input AD instances as a 2D array and save as self.der

Class AD_System will also implement the following functions, very similar to the simple AD class:
1. \__add__ & \__radd__
2. \__sub__ & \__rsub__
3. \__mul__ & \__rmul__
4. \__div__ & \__rdiv__
5. \__pow__
6. \__abs__
7. sin
8. cos
9. tan
10. exp
11. log
12. get_val()
13. get_der(i=1)
14. \__eq__
15. \__str__

## External Dependencies

We will need **numpy** for impletation and **pytest** and **doctest** for testing.